# create_csv_Linesearch_by_extension

**What it does**: 
- searches for numbers within all files in directory with specified extension `findending` e.g. within an output file of a simulation

1. Search for lines (Parameter: `findtext`)
2. Within the line the number will be found between `findvalue_start` and `findvalue_end`
3. The data will be saved to `data.csv` and the labels can be renamed from the filename between `filename_to_label_start` and `filename_to_label_end`

In [ ]:
# import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re  # for finding text in files
import glob # for loop over all extensions


# variables

## find all fines with *.
findending = ".out"

## text so search for
findtext = ["stress =",
"displacement ="]
findvalue_end = ["MPa",
"mm"]
findvalue_start = "="    # start marker for number recognition
filename_to_label_start = "" # start marker for label from filename 
filename_to_label_end = ".out" # end marker for label from filename 


# get list of filenames
filenames = []
labels = []

for filename in glob.iglob(r"*"+findending):
    filenames.extend([filename])
    # cut string
    start = filename.find(filename_to_label_start) + len(filename_to_label_start)
    end = filename.find(filename_to_label_end)
    substring = filename[start:end]
    labels.extend([substring])

# number of filenames
number_batches=len(filenames)

# dataframe + colums
cols = []
for i in range(len(findtext)): 
    col = findtext[i]+" ["+findvalue_end[i]+"]" # columns with units
    cols.extend([col])

# dataframe
df = pd.DataFrame(columns = cols, index=labels) 

# loop for all filenames
for i in range(number_batches):

    # read file
    textfile = open(filenames[i], 'r')
    filetext = textfile.read()
    
    # read value
    for j in range(len(findtext)):
        # get string line
        findtextend = ".*$"
        fullstring = re.findall(findtext[j]+ findtextend, filetext,re.MULTILINE)[0]

        # cut string
        start = fullstring.find(findvalue_start) + len(findvalue_start)
        end = fullstring.find(findvalue_end[j])
        substring = fullstring[start:end]
        df.loc[labels[i], cols[j]] = float(substring)


#save
df.to_csv('data.csv', index=True)